In [ ]:
import os

import pandas as pd

In [ ]:
ann_pru = pd.read_csv(
    '../../../données/HELIOS_2022_06_24/ANN_RPU_2022_06_23.CSV',
    sep=';',
    encoding='latin1'
)

In [ ]:
ann_pru.head(2)

Données dupliqués:

In [ ]:
dupliques = ann_pru.duplicated(subset=['Finess', 'Année'], keep=False)

In [ ]:
dupliques.sum()

In [ ]:
dupliques = ann_pru.duplicated(subset=['Finess', 'Année'], keep=False)

In [ ]:
dupliques.sum()